In [1]:
import sys
sys.path.append('../kkchuchu.mushroom/')

import tensorflow as tf

from Ramp import Ramp
from Sink import ToCommaCsv

In [2]:
(TEST_FILLE, TRAIN_FILE, SAMPLE_FILE, SUBMIT_FILE) = (
    '../../datasets/Allstate Claims Severity/test.csv', 
    '../../datasets/Allstate Claims Severity/train.csv', 
    '../../datasets/Allstate Claims Severity/Sample_Submission.csv',
    '../../datasets/Allstate Claims Severity/Submission.csv',
)
UID, TARGET = ['id'], ['loss']
CAT = ['cat%i' % i for i in range(1, 117)]
CONT = ['cont%i' % i for i in range(1, 15)]
DIMENSION = len(CAT) + len(CONT)

In [3]:
column_list = Ramp(TRAIN_FILE).sample().columns.tolist()

In [4]:
Ramp(TRAIN_FILE).sample()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,...,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,...,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85
5,13,A,B,A,A,A,A,A,A,B,...,0.364464,0.401162,0.26847,0.46226,0.50556,0.366788,0.359249,0.345247,0.726792,5142.87
6,14,A,A,A,A,B,A,A,A,A,...,0.381515,0.363768,0.24564,0.40455,0.47225,0.334828,0.352251,0.342239,0.382931,1132.22
7,20,A,B,A,B,A,A,A,A,B,...,0.867021,0.583389,0.90267,0.84847,0.80218,0.644013,0.785706,0.859764,0.242416,3585.75
8,23,A,B,B,B,B,A,A,A,B,...,0.628534,0.384099,0.61229,0.38249,0.51111,0.682315,0.669033,0.756454,0.361191,10280.20
9,24,A,B,A,A,B,B,A,A,B,...,0.713343,0.469223,0.30260,0.67135,0.83510,0.863052,0.879347,0.822493,0.294523,6184.59


In [5]:
Ramp(SAMPLE_FILE).sample()

,id,loss
0,4,0
1,6,0
2,9,0
3,12,0
4,15,0
5,17,0
6,21,0
7,28,0
8,32,0
9,43,0


In [6]:
def Preprocess(df, y_d=True, y=None):
    def toint(str):
        sum = 0.
        for i, char in enumerate(str):
            sum += ord(char) * pow(1000, i)
        return sum
    for column in CAT:
        df[column] = df[column].apply(toint)
    if y_d:
        y = df[TARGET].as_matrix()
    return df[UID].as_matrix(), df[CAT + CONT].as_matrix(), y

In [7]:
# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
log_path = "../../datasets/temp/tensorflow_logs/allstate_claims_severity"

# Network Parameters
n_hidden_1 = DIMENSION # 1st layer number of features
n_hidden_2 = DIMENSION # 2nd layer number of features
n_hidden_3 = DIMENSION # 3nd layer number of features
n_input = DIMENSION # MNIST data input (img shape: 28*28)
n_classes = 1

# tf Graph input
x = tf.placeholder("float", [None, n_input], name="InputData")
y = tf.placeholder("float", [None, 1], name="OutputResult")

In [8]:
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1_drop = tf.nn.dropout(layer_1, 0.4)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1_drop, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_2_drop = tf.nn.dropout(layer_2, 0.2)
    
    layer_3 = tf.add(tf.matmul(layer_2_drop, weights['h3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3)
    layer_3_drop = tf.nn.dropout(layer_3, 0.2)
    
    # Output layer with linear activation
    out_layer = tf.matmul(layer_3_drop, weights['out']) + biases['out']
    return out_layer

In [9]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_hidden_3, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer: Mean Absolute Error (MAE), Adadelta
cost = tf.reduce_mean(tf.abs(tf.sub(pred, y)))
optimizer = tf.train.AdadeltaOptimizer(learning_rate=learning_rate).minimize(cost)

In [10]:
# Initializing the variables
init = tf.global_variables_initializer()
NUM_THREADS = 1
# Launch the graph
# sess = tf.InteractiveSession(config=tf.ConfigProto(intra_op_parallelism_threads=NUM_THREADS))
sess = tf.Session()
sess.run(init)

In [11]:
train_stream = (Preprocess(record) for record in Ramp(TRAIN_FILE).stream(batchsize=1))

In [16]:
tf.scalar_summary("loss", cost)

summary_writer = tf.train.SummaryWriter(log_path, graph=tf.get_default_graph())
# Merge all summaries into a single op
merged_summary_op = tf.merge_all_summaries()

In [17]:
# Training cycle
total_cost = 0.
for epoch in range(100):
    uid, rx, ry = next(train_stream)
    _, c, summary = sess.run([optimizer, cost, merged_summary_op], 
                             feed_dict={x: rx, y: ry})
    summary_writer.add_summary(summary, epoch)
    
    #Compute total cost
    total_cost += c
print("Optimization Finished!")
 
epoch, total_cost

Optimization Finished!


(99, 49759458154.875)

In [ ]:
sink = ToCommaCsv(SUBMIT_FILE)
# Using trainned model to predict test file
test_pool = (Preprocess(row, y_d=False) for row in Ramp(TEST_FILLE).stream())
counter = 0
for uid, x_input, _ in test_pool:
    counter += 1
    r = sess.run(pred, feed_dict={x:x_input})
    sink.add([uid[0][0], r[0][0]])
    if counter % 10000 is 0:
        print(counter)
del sink
counter

In [19]:
sess.close()

In [ ]:
"""
run docker exec -i mario tensorboard --logdir='/home/jovyan/work/datasets/temp/tensorflow_logs/allstate_claims_severity'
and open browser localhost:6006
"""